In [1]:
import tensorflow as tf
import tensorflow_text as text
import functools
print("TensorFlow version: ", tf.__version__)


2023-08-31 14:10:47.417126: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-31 14:10:47.494697: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


TensorFlow version:  2.11.0


In [3]:
examples = {
    "text_a": [
      "Sponge bob Squarepants is an Avenger",
      "Marvel Avengers"
    ],
    "text_b": [
     "Barack Obama is the President.",
     "President is the highest office"
  ],
}

dataset = tf.data.Dataset.from_tensor_slices(examples)
next(iter(dataset))


2023-08-31 14:11:09.215404: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-08-31 14:11:09.227401: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-08-31 14:11:09.228334: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-08-31 14:11:09.230506: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuil

{'text_a': <tf.Tensor: shape=(), dtype=string, numpy=b'Sponge bob Squarepants is an Avenger'>,
 'text_b': <tf.Tensor: shape=(), dtype=string, numpy=b'Barack Obama is the President.'>}

In [4]:
_VOCAB = [
    # Special tokens
    b"[UNK]", b"[MASK]", b"[RANDOM]", b"[CLS]", b"[SEP]",
    # Suffixes
    b"##ack", b"##ama", b"##ger", b"##gers", b"##onge", b"##pants",  b"##uare",
    b"##vel", b"##ven", b"an", b"A", b"Bar", b"Hates", b"Mar", b"Ob",
    b"Patrick", b"President", b"Sp", b"Sq", b"bob", b"box", b"has", b"highest",
    b"is", b"office", b"the",
]

_START_TOKEN = _VOCAB.index(b"[CLS]")
_END_TOKEN = _VOCAB.index(b"[SEP]")
_MASK_TOKEN = _VOCAB.index(b"[MASK]")
_RANDOM_TOKEN = _VOCAB.index(b"[RANDOM]")
_UNK_TOKEN = _VOCAB.index(b"[UNK]")
_MAX_SEQ_LEN = 8
_MAX_PREDICTIONS_PER_BATCH = 5

_VOCAB_SIZE = len(_VOCAB)

lookup_table = tf.lookup.StaticVocabularyTable(
    tf.lookup.KeyValueTensorInitializer(
      keys=_VOCAB,
      key_dtype=tf.string,
      values=tf.range(
          tf.size(_VOCAB, out_type=tf.int64), dtype=tf.int64),
          value_dtype=tf.int64
        ),
      num_oov_buckets=1
)


In [29]:
segments_combined, segments_ids = text.combine_segments(
  trimmed,
  start_of_sequence_id=_START_TOKEN, end_of_segment_id=_END_TOKEN)
segments_combined, segments_ids

(<tf.RaggedTensor [[3, 22, 9, 24, 23, 4, 16, 5, 19, 6, 4],
  [3, 18, 12, 15, 13, 4, 21, 28, 30, 27, 4]]>,
 <tf.RaggedTensor [[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1],
  [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1]]>)

In [18]:
random_selector = text.RandomItemSelector(
    max_selections_per_batch=_MAX_PREDICTIONS_PER_BATCH,
    selection_rate=0.2,
    unselectable_ids=[_START_TOKEN, _END_TOKEN, _UNK_TOKEN]
)

In [23]:
mask_values_chooser = text.MaskValuesChooser(_VOCAB_SIZE, _MASK_TOKEN, 0.8)


In [24]:
def bert_pretrain_preprocess(vocab_table, features):
    # Input is a string Tensor of documents, shape [batch, 1].
    text_a = features["text_a"]
    text_b = features["text_b"]

    # Tokenize segments to shape [num_sentences, (num_words)] each.
    tokenizer = text.BertTokenizer(
      vocab_table,
      token_out_type=tf.int64)
    segments = [tokenizer.tokenize(text).merge_dims(
      1, -1) for text in (text_a, text_b)]

    # Truncate inputs to a maximum length.
    trimmer = text.RoundRobinTrimmer(max_seq_length=6)
    trimmed_segments = trimmer.trim(segments)

    # Combine segments, get segment ids and add special tokens.
    segments_combined, segment_ids = text.combine_segments(
      trimmed_segments,
      start_of_sequence_id=_START_TOKEN,
      end_of_segment_id=_END_TOKEN)

    # Apply dynamic masking task.
    masked_input_ids, masked_lm_positions, masked_lm_ids = (
      text.mask_language_model(
        segments_combined,
        random_selector,
        mask_values_chooser,
      )
    )

    # Prepare and pad combined segment inputs
    input_word_ids, input_mask = text.pad_model_inputs(
    masked_input_ids, max_seq_length=_MAX_SEQ_LEN)
    input_type_ids, _ = text.pad_model_inputs(
    segment_ids, max_seq_length=_MAX_SEQ_LEN)

    # Prepare and pad masking task inputs
    masked_lm_positions, masked_lm_weights = text.pad_model_inputs(
    masked_lm_positions, max_seq_length=_MAX_PREDICTIONS_PER_BATCH)
    masked_lm_ids, _ = text.pad_model_inputs(
    masked_lm_ids, max_seq_length=_MAX_PREDICTIONS_PER_BATCH)

    model_inputs = {
      "input_word_ids": input_word_ids,
      "input_mask": input_mask,
      "input_type_ids": input_type_ids,
      "masked_lm_ids": masked_lm_ids,
      "masked_lm_positions": masked_lm_positions,
      "masked_lm_weights": masked_lm_weights,
    }
    return model_inputs

In [25]:
dataset = (
    tf.data.Dataset.from_tensors(examples)
    .map(functools.partial(bert_pretrain_preprocess, lookup_table))
)

next(iter(dataset))


{'input_word_ids': <tf.Tensor: shape=(2, 8), dtype=int64, numpy=
 array([[ 3,  1,  9, 24,  4, 16,  5,  1],
        [ 3, 18, 12, 15,  4, 21, 28,  1]])>,
 'input_mask': <tf.Tensor: shape=(2, 8), dtype=int64, numpy=
 array([[1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1]])>,
 'input_type_ids': <tf.Tensor: shape=(2, 8), dtype=int64, numpy=
 array([[0, 0, 0, 0, 0, 1, 1, 1],
        [0, 0, 0, 0, 0, 1, 1, 1]])>,
 'masked_lm_ids': <tf.Tensor: shape=(2, 5), dtype=int64, numpy=
 array([[22, 19,  0,  0,  0],
        [28, 30,  0,  0,  0]])>,
 'masked_lm_positions': <tf.Tensor: shape=(2, 5), dtype=int64, numpy=
 array([[1, 7, 0, 0, 0],
        [6, 7, 0, 0, 0]])>,
 'masked_lm_weights': <tf.Tensor: shape=(2, 5), dtype=int64, numpy=
 array([[1, 1, 0, 0, 0],
        [1, 1, 0, 0, 0]])>}

In [122]:
class BertEmbedding(tf.keras.layers.Layer):
    """Bert embedding is composed of a positional embedding layer, a segment embedding layer and a normal embedding layer

    Input shape
      - token index 2D tensor with shape: ``(batch_size, sequence_length)``.
      - segment index 2D tensor with shape: ``(batch_size, sequence_length)``.

    Output shape
      - 3D tensor with shape: ``(batch_size, sequence_length, embedding_size)``.

    References
        - [BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding](https://arxiv.org/pdf/1810.04805.pdf)
    """

    def __init__(self, vocab_size, seq_length=128, dim=512, **kwargs):
        super(BertEmbedding, self).__init__(**kwargs)
        assert seq_length % 2 == 0, "Output dimension needs to be an even integer"
        self.length = seq_length
        self.dim = dim
        self.token_emb = tf.keras.layers.Embedding(
            input_dim=vocab_size, output_dim=dim, mask_zero=True
        )
        self.position_emb = tf.keras.layers.Embedding(input_dim=seq_length, output_dim=dim)
        self.segment_emb = tf.keras.layers.Embedding(input_dim=2, output_dim=dim)

    def call(self, inputs, **kwargs):
        tokens = inputs["input_word_ids"]
        segments = inputs["input_type_ids"]
        length = tf.shape(tokens)[1]
        embedded_tokens = self.token_emb(tokens)
        embedded_segments = self.segment_emb(segments)
        embedded_positions = self.position_emb(tf.range(length))
        # This factor sets the relative scale of the embedding and positonal_encoding.
        embedded_tokens *= tf.math.sqrt(tf.cast(self.dim, tf.float32))
        return (
            embedded_tokens + embedded_segments + embedded_positions[tf.newaxis, :, :]
        )

    # Pass mask from token_emb, https://www.tensorflow.org/guide/keras/understanding_masking_and_padding#supporting_masking_in_your_custom_layers
    def compute_mask(self, inputs, mask=None):
        return inputs["input_mask"]


In [123]:
bert = BertEmbedding(len(_VOCAB), _MAX_SEQ_LEN)

In [127]:
bert(next(iter(dataset)))

<tf.Tensor: shape=(2, 8, 512), dtype=float32, numpy=
array([[[ 0.05752446,  0.75627196,  0.27433577, ...,  1.0190184 ,
         -0.83793116, -0.3768208 ],
        [ 0.04593604, -0.8231284 ,  1.1117262 , ...,  0.26253074,
         -0.8496783 , -0.8961436 ],
        [-0.11777939,  0.39778906,  0.95789903, ..., -0.38639542,
          0.64713544, -0.4776726 ],
        ...,
        [-0.61908835, -0.43666652,  0.6317425 , ...,  0.34010994,
         -0.7603367 ,  1.1500205 ],
        [ 0.9602606 ,  0.16026199, -0.7066235 , ...,  0.38239995,
         -0.60343444,  1.05717   ],
        [-0.75959325, -0.202389  ,  1.0904251 , ...,  0.01723169,
         -0.53571177, -0.68253744]],

       [[ 0.05752446,  0.75627196,  0.27433577, ...,  1.0190184 ,
         -0.83793116, -0.3768208 ],
        [-0.59359604, -0.35077828,  0.62417734, ...,  0.4131376 ,
         -0.7492717 ,  1.0248871 ],
        [-0.61279154,  0.91318214,  1.0679604 , ...,  1.039956  ,
         -0.2652515 ,  0.6798283 ],
        ...,
 

In [116]:
%load_ext autoreload
%autoreload 2
from trainer.models.common.transformer import Encoder

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [141]:
class Bert(tf.keras.layers.Layer):
    """Bert model is a stack of multiple layers of encoders

    Input shape
      - 3D tensor with shape: ``(batch_size, sequence_length, embedding_size)``.

    Output shape
      - 3D tensor with shape: ``(batch_size, sequence_length, embedding_size)``.

    References
        - [BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding](https://arxiv.org/pdf/1810.04805.pdf)
    """

    def __init__(
        self,
        vocab_size,
        seq_length,
        layer_num=6,
        model_dim=512,
        ff_dim=2048,
        dropout=0.1,
        head_num=8,
        **kwargs,
    ):
        super(Bert, self).__init__(**kwargs)
        self.emb = BertEmbedding(
            vocab_size=vocab_size, seq_length=seq_length, dim=model_dim
        )
        self.encoders = [
            Encoder(
                model_dim=model_dim, ff_dim=ff_dim, dropout=dropout, head_num=head_num
            )
            for _ in range(layer_num)
        ]

    def call(self, inputs, training=False):
        # shape [batch_size, token_length]
        emb = self.emb(inputs, training=training)
        # shape [batch_size, token_length, model_dim]
        for encoder in self.encoders:
            emb = encoder(emb, training=training)
        # shape [batch_size, token_length, model_dim]
        return emb


class BertMLM(tf.keras.layers.Layer):
    """Masked language model simply mask some percentage of the input tokens at random, and then predict those masked tokens

    Input shape
      - 3D tensor with shape: ``(batch_size, sequence_length, embedding_size)``.

    Output shape
      - 3D tensor with shape: ``(batch_size, sequence_length, embedding_size)``.

    References
        - [BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding](https://arxiv.org/pdf/1810.04805.pdf)
    """

    def __init__(
        self,
        vocab_size,
        seq_length,
        layer_num=6,
        model_dim=512,
        ff_dim=2048,
        dropout=0.1,
        head_num=8,
        **kwargs,
    ):
        super(BertMLM, self).__init__(**kwargs)

        self.emb = Bert(
            vocab_size=vocab_size,
            seq_length=seq_length,
            layer_num=layer_num,
            model_dim=model_dim,
            ff_dim=ff_dim,
            dropout=dropout,
            head_num=head_num,
            **kwargs,
        )
        self.dropout = tf.keras.layers.Dropout(dropout)
        self.dense = tf.keras.layers.Dense(
            vocab_size, activation=tf.keras.activations.softmax
        )

    def call(self, inputs, training=False):
        # shape [batch_size, token_length, model_dim]
        emb = self.dropout(self.emb(inputs, training=training))
        # shape [batch_size, token_length, vocab_size_logits]
        emb = self.dense(emb)
        # gather the corresponding logits per the masked_lm_positions
        return tf.gather(emb, inputs["masked_lm_positions"], axis=1, batch_dims=1)


class BertNSP(tf.keras.layers.Layer):
    """Next sentence predition use the CLS token embedding for prediction

    Input shape
      - 3D tensor with shape: ``(batch_size, sequence_length, embedding_size)``.

    Output shape
      - 3D tensor with shape: ``(batch_size, 1)``.

    References
        - [BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding](https://arxiv.org/pdf/1810.04805.pdf)
    """

    def __init__(
        self,
        vocab_size,
        seq_length,
        layer_num=6,
        model_dim=512,
        ff_dim=2048,
        dropout=0.1,
        head_num=8,
        **kwargs,
    ):
        super(BertNSP, self).__init__(**kwargs)

        self.emb = Bert(
            vocab_size=vocab_size,
            seq_length=seq_length,
            layer_num=layer_num,
            model_dim=model_dim,
            ff_dim=ff_dim,
            dropout=dropout,
            head_num=head_num,
            **kwargs,
        )
        self.dropout = tf.keras.layers.Dropout(dropout)
        self.dense = tf.keras.layers.Dense(2, activation=tf.keras.activations.softmax)

    def call(self, inputs, training=False):
        # shape [batch_size, token_length, model_dim]
        emb = self.dropout(self.emb(inputs, training=training))
        # shape [batch_size, 2]
        return self.dense(emb[:, 0, :])


In [153]:
mlm = BertMLM(len(_VOCAB), _MAX_SEQ_LEN)
mlm(next(iter(dataset)))

<tf.Tensor: shape=(2, 5, 31), dtype=float32, numpy=
array([[[0.0104665 , 0.02893532, 0.00224435, 0.00421711, 0.05016666,
         0.0070851 , 0.02275247, 0.02934286, 0.03873889, 0.05066179,
         0.0135875 , 0.00433905, 0.02290054, 0.0051624 , 0.19703811,
         0.01757844, 0.00519997, 0.00181911, 0.00170542, 0.02262172,
         0.01330194, 0.00624211, 0.14401814, 0.01847748, 0.00532914,
         0.00253174, 0.00186335, 0.02260299, 0.22262819, 0.01252946,
         0.01391215],
        [0.01009972, 0.02907079, 0.00230911, 0.00433553, 0.05026842,
         0.00702415, 0.0230408 , 0.02889612, 0.03803893, 0.05250223,
         0.01317527, 0.0045732 , 0.02294951, 0.00522047, 0.20489928,
         0.01718402, 0.00518294, 0.00172164, 0.00181506, 0.02320082,
         0.01289726, 0.006328  , 0.14048722, 0.01804808, 0.00526207,
         0.00262906, 0.00182247, 0.02258459, 0.21790743, 0.01211095,
         0.01441482],
        [0.01121071, 0.04354887, 0.00489386, 0.00732061, 0.05677856,
       

In [145]:
nsp = BertNSP(len(_VOCAB), _MAX_SEQ_LEN)
nsp(next(iter(dataset)))

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[0.33159786, 0.66840214],
       [0.36663258, 0.6333674 ]], dtype=float32)>

In [180]:
def masked_loss(label, pred):
    mask = label != 0
    loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=False, reduction="none"
    )
    loss = loss_object(label, pred)
    
    # mask indices where label == 0 (padding)
    mask = tf.cast(mask, dtype=loss.dtype)
    loss *= mask

    loss = tf.reduce_sum(loss) / tf.reduce_sum(mask)
    return loss


def masked_accuracy(label, pred):
    # get the prediction index for target token
    pred = tf.argmax(pred, axis=2)
    label = tf.cast(label, pred.dtype)
    match = label == pred

    mask = label != 0

    match = match & mask

    match = tf.cast(match, dtype=tf.float32)
    mask = tf.cast(mask, dtype=tf.float32)
    return tf.reduce_sum(match) / tf.reduce_sum(mask)


In [181]:
class BertModel(tf.keras.Model):

    def call(self, inputs, training=False) -> tf.Tensor:
        logits = mlm(inputs, training=training)
        try:
            # Drop the keras mask, so it doesn't scale the losses/metrics.
            # b/250038731
            del logits._keras_mask
        except AttributeError:
            pass
        return logits


In [182]:
bert_model = BertModel()

In [187]:
bert_model.compile(
            loss=masked_loss,
            optimizer=tf.keras.optimizers.Adam(
                learning_rate=0.0001,
            ),
            metrics=[masked_accuracy],
        )

In [188]:
bert_model.fit(dataset.repeat(1000).map(lambda x: (x, x["masked_lm_ids"])))

1000/1000 [==============================] - 13s 11ms/step - loss: 3.9962 - masked_accuracy: 0.0512
